In [1]:
from hmmlearn import hmm
import pandas as pd

## First get our data in 

In [150]:
data = pd.read_csv('../Protein_Prediction/sequences.csv').drop('Unnamed: 0', axis=1)
labels = pd.read_csv('../Protein_Prediction/5_formatted_labels.csv').drop('Unnamed: 0', axis=1)
labels.columns = ['State']

In [151]:
data.head()

,Amino Acid
0,V
1,L
2,S
3,P
4,A


In [152]:
data.values[:6]

array([['V'],
       ['L'],
       ['S'],
       ['P'],
       ['A'],
       ['D']], dtype=object)

In [153]:
labels.head()

,State
0,5.0
1,5.0
2,0.0
3,0.0
4,0.0


In [154]:
labels = labels.values.reshape(1, -1).ravel()

In [155]:
# some silly formatting
labels = [5.0] + list(labels)

In [156]:
labels

[5.0,
 5.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 2.0,
 2.0,
 3.0,
 5.0,
 5.0,
 3.0,
 2.0,
 2.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 2.0,
 2.0,
 3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.0,
 2.0,
 2.0,
 2.0,
 5.0,
 5.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 2.0,
 2.0,
 5.0,
 5.0,
 5.0,
 5.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0,
 4.0,
 3.0,
 3.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [157]:
print data.shape
print len(labels)

(134815, 1)
134815


## Now calculate the transition matrix

In [159]:
from collections import defaultdict
# calculate transitions
trans = defaultdict(int)

for i in xrange(len(labels)-1):
    
    if labels[i] not in trans:
        trans[labels[i]] = defaultdict(int)
    
    trans[labels[i]][labels[i+1]] += 1

In [160]:
t_mat = pd.DataFrame(trans)
t_mat

,0.0,1.0,2.0,3.0,4.0,5.0
0.0,38210,197,655,768,100,2751
1.0,124,14983,656,843,49,2081
2.0,2392,636,6171,631,191,2068
3.0,747,566,1503,3260,157,3038
4.0,74,65,219,217,42,748
5.0,1134,2289,2885,3552,826,39986


In [53]:
# states = ['H', 'E', 'T', 'S', 'B', 'U']
# t_mat.columns = states
# t_mat.index = states

In [56]:
t_mat.head()

,H,E,T,S,B,U
H,38210,197,655,768,100,2751
E,124,14983,656,843,49,2081
T,2392,636,6171,631,191,2068
S,747,566,1503,3260,157,3038
B,74,65,219,217,42,748


In [57]:
t_mat.to_csv('transition.mat')

In [58]:
t_mat.values

array([[38210,   197,   655,   768,   100,  2751],
       [  124, 14983,   656,   843,    49,  2081],
       [ 2392,   636,  6171,   631,   191,  2068],
       [  747,   566,  1503,  3260,   157,  3038],
       [   74,    65,   219,   217,    42,   748],
       [ 1134,  2289,  2885,  3552,   826, 39986]])

## Now calculate the emission matrix

In [165]:
amino_acids = list(data.values.reshape(1, -1).ravel())
amino_acids[:5]

['V', 'L', 'S', 'P', 'A']

In [172]:
emis = defaultdict(int)

for l, aa in zip(labels, amino_acids):
    
    if l not in emis:
        emis[l] = defaultdict(int)
    
    emis[l][aa] += 1
    

In [171]:
emis_df = pd.DataFrame(emis)
# we know they're really just zeros
emis_df.fillna(value=0.0, inplace=True)
emis_df.columns = states
emis_df

,H,E,T,S,B,U
-,1,0.0,0.0,0.0,0.0,30095
A,4949,1097.0,749.0,573.0,50.0,1430
C,392,318.0,105.0,82.0,18.0,240
D,2151,620.0,968.0,706.0,62.0,1773
E,3750,864.0,823.0,577.0,50.0,989
F,1797,1124.0,327.0,337.0,101.0,755
G,1623,958.0,2000.0,1299.0,98.0,1582
H,887,445.0,373.0,279.0,40.0,568
I,2567,1963.0,256.0,335.0,118.0,851
K,2566,926.0,748.0,542.0,51.0,1014


In [173]:
emis_df = pd.DataFrame(emis)
# we know they're really just zeros
emis_df.fillna(value=0.0, inplace=True)
emis_df.index = [float(x) for x in xrange(len(emis_df.index))]
emis_df

,0.0,1.0,2.0,3.0,4.0,5.0
0.0,1,0.0,0.0,0.0,0.0,30095
1.0,4949,1097.0,749.0,573.0,50.0,1430
2.0,392,318.0,105.0,82.0,18.0,240
3.0,2151,620.0,968.0,706.0,62.0,1773
4.0,3750,864.0,823.0,577.0,50.0,989
5.0,1797,1124.0,327.0,337.0,101.0,755
6.0,1623,958.0,2000.0,1299.0,98.0,1582
7.0,887,445.0,373.0,279.0,40.0,568
8.0,2567,1963.0,256.0,335.0,118.0,851
9.0,2566,926.0,748.0,542.0,51.0,1014


# Finally, construct the HMM

In [78]:
import numpy as np
from collections import Counter

In [178]:
# getting the starting probability 
start_prob = Counter(labels)
total = sum(start_prob.values())

start_prob = {k: v*1.0/total for k, v in start_prob.items()}
start_prob = start_prob.values()
start_prob = np.asarray(start_prob).reshape(-1, )
start_prob

array([ 0.3165894 ,  0.13897563,  0.08967103,  0.06876831,  0.01012499,
        0.37587064])

In [175]:
features = sorted(list(set(list(data.values.ravel()))))
features

['-',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y']

In [179]:
feature_map = {k: i for i, k in enumerate(features)}

In [180]:
sequence = [feature_map[aa] for aa in data.values.ravel()]
sequence = np.asarray(sequence).reshape(-1, 1)
sequence[:5]

array([[18],
       [10],
       [16],
       [13],
       [ 1]])

In [194]:
# encode transition matrix
trans_mat = np.asarray([l*1.0/sum(l) for l in t_mat.values])
trans_mat.shape

(6, 6)

In [193]:
# encode emission matrix
emiss_mat = np.asarray([r*1.0/sum(r) for r in emis_df.values.T])
emiss_mat.shape

(6, 21)

In [195]:
model = hmm.MultinomialHMM(n_components=6, init_params='est')

In [205]:
# fixing some of the attributes
model.startprob_prior = start_prob
model.transmat_prior = trans_mat
model.emissionprob_ = emiss_mat
model.n_features = len(features)

In [202]:
model.fit(sequence)

MultinomialHMM(algorithm='viterbi', init_params='est', n_components=6,
        n_iter=10, params='ste',
        random_state=<mtrand.RandomState object at 0x10497b870>,
        startprob_prior=array([ 0.31659,  0.13898,  0.08967,  0.06877,  0.01012,  0.37587]),
        tol=0.01,
        transmat_prior=array([[ 0.89525,  0.00462,  0.01535,  0.01799,  0.00234,  0.06445],
       [ 0.00662,  0.79969,  0.03501,  0.04499,  0.00262,  0.11107],
       [ 0.19787,  0.05261,  0.51046,  0.0522 ,  0.0158 ,  0.17106],
       [ 0.08057,  0.06105,  0.16212,  0.35163,  0.01693,  0.32769],
       [ 0.05421,  0.04762,  0.16044,  0.15897,  0.03077,  0.54799],
       [ 0.02238,  0.04517,  0.05693,  0.0701 ,  0.0163 ,  0.78911]]),
        verbose=False)

In [206]:
preds = model.predict(sequence)

In [207]:
# how well are we predicting
print np.mean([1.0 if y_p==y_i else 0.0 for y_p, y_i in zip(preds, labels)])

0.292000148351
